# Задание 1 #

Выполнено в PostgreSQL

## 1. ##  
Посчитать % изменение количества клиентов, совершивших покупку, месяц-к-месяцу.

Заголовки колонок: client_id, order_id, order_date. Название таблицы - orders.


~~~~sql
SELECT
    unique_users,
    -- через окно упорядоченное по месяцам считаем % изменения количества клиентов
    ROUND(
        (unique_users - LAG(unique_users, 1) OVER w)*100.0 / unique_users
        ) AS diff
FROM (
    SELECT
        COUNT(DISTINCT user_id) as unique_users, -- считаем количество уникальных id, совершивших покупку, в месяц
        DATE_TRUNC('month', CAST(order_date as date)) as order_date --приводим к формату даты, оставляем вид "год-месяц"
    FROM orders
    GROUP BY order_date
    ) AS tmp
WINDOW w AS (ORDER BY order_date)
ORDER BY order_date
LIMIT 1000
~~~~

## 2. ##
Вывести сумму GMV (Gross Merchandise Value) с нарастающим итогом по дням.

Заголовки колонок: fact_date, gmv. Название таблицы - revenue. Предполагаем, что в таблице указаны только уникальные даты.

1) если не нужно разбивать по месяцам/годам
~~~~sql		
SELECT
    fact_date,
    SUM(gmv) OVER w AS gmv_cumulative -- считаем сумму GMV с нарастающим итогом по дням через окно
FROM revenue
WINDOW w AS (ORDER BY fact_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
ORDER BY fact_date
LIMIT 1000
~~~~

2) если нужно разбивать подсчет. Например, считать нарастающий итог отдельно по каждому году.

~~~~sql
-- приводим дату к типу даты (если вдруг это не так), создаем столбец с годом
WITH tmp as (
    SELECT EXTRACT('year' FROM CAST(fact_date as date)) AS fact_year,
        fact_date,
        gmv
    from revenue)
SELECT
	fact_year,
    fact_date,
	-- считаем сумму GMV с нарастающим итогом по дням через окно для каждого года в отдельности
    SUM(gmv) OVER w AS gmv_cumulative 
FROM tmp
WINDOW w AS (PARTITION BY fact_year
			ORDER BY fact_year, fact_date
			ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
ORDER BY sale_date
LIMIT 1000
~~~~

## 3. ##
Получить время отклика на каждое письмо (письмо идентифицируется по полю mail_id), отправленное пользователем mr_employee@ozon.ru.

Дана таблица с логом электронных писем пользователя mr_employee@ozon.ru (т.е. письма, отправленные с этой электронной почты и полученные на нее). У всех цепочек сообщений уникальная тема. В одной цепочке может быть несколько писем.

Заголовки колонок: 
- mail_id	- идентификатор письма
- mail_from - отправитель
- mail_to	- получатель
- mail_subject - тема письма
- timestamp - метка времени отправки письма (изменяю в звпросе на reply, чтобы не было конфликта названия колонки и типа данных)

Название таблицы - mailbox.

~~~~sql
SELECT
    mail_id, mail_from, mail_to, mail_subject, reply, reply_time, flag
FROM
    (
    SELECT
        mail_id, mail_from, mail_to, mail_subject, reply,
        -- окно имеет секцию равную уникальной теме и фрейм "текущая строка-следующая строка"
        -- сортировка внутри тем по времени отправки писем
        -- вычитаем из времени следующего по порядку в группе письма время текущей строки (письма)
        -- получаем время отклика на каждое письмо
        lead(reply, 1) over w - reply  AS reply_time,
        -- поставим флажок 0, если отправитель в текущей строке, равен отправителю в следующей строке
        -- флажок 1 для всех остальных, это будут строки, которые нам нужны
        CASE WHEN mail_from  = lead(mail_from) over w THEN 0 ELSE 1 END as flag
    FROM mailbox1
    WINDOW w AS (PARTITION BY mail_subject
            ORDER BY reply
            ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)
    ORDER BY mail_subject, reply
    ) as reply_counter
-- берем только письма, отправленные mr_employee@ozon.ru с флажком 1
-- получаем для каждого mail_id в колонке reply_time время отклика на него
WHERE mail_from = 'mr_employee@ozon.ru' AND flag = 1
ORDER BY mail_subject, reply
LIMIT 100
~~~~

## 4. ##
Вывести id сотрудников с разницей в заработной плате в пределах 5000 рублей

Заголовки колонок: 
- employee_id	- идентификатор сотрудника
- salary_rub - зарплата

Название таблицы - salary.

~~~~sql
SELECT
    employee_id,
    -- приводим колонку employee_id к строковому типу
    -- собираем в одну строку все id по окну с range -/+ 5000
    STRING_AGG(CAST(employee_id AS varchar), ', ') OVER w AS emp_list,
    MIN(salary_rub) OVER w AS lower_bound, -- выводим нижнюю границу з/п для группы id
    MAX(salary_rub) OVER w AS upper_bound -- выводим верхнюю границу з/п для группы id
FROM salary
WINDOW w AS (ORDER BY salary_rub
            RANGE BETWEEN 5000 preceding AND 5000 following)
LIMIT 100
-- получаем для каждого employee_id группу сотрудников, которые отличаются по з/п от данного сотрудника на 5000
~~~~

# Задание 2  - Case Study. Опыт продавца#

1. Нарисовать процесс регистрации и выхода продавца на Маркетплейс от момента регистрации до 1-ой продажи по схеме DBS и описать основные отличия от схемы FBO и FBS

2. Описать основные сложности, которые могут возникнуть на каждом этапе с момента регистрации и первой продажи по схеме DBS

Процесс и сложности описаны в документе по сслыке:

https://drive.google.com/file/d/12vHFn4VQd3l9taFGpMum_t03Jy3A9luM/view?usp=sharing

3. Разработать метрики, которые на ваш взгляд могут показать ключевые отличия в процессах между указанными выше схемами работы;

4. Установить нормативные значения (KPI) по метрикам и обосновать релевантность и реалистичность этих KPI

Метрики и KPI описаны в документе по ссылке:

https://docs.google.com/document/d/1CH7PT_CkODaL6cQtJaYwo0TyRG0rk24a/edit?usp=sharing&ouid=100679202842882263816&rtpof=true&sd=true

# Задание 3 #

Для работы загружаю данные из Clickhouse в Jupiter, обрабатываю в python, визуализацию делаю в tableau.

In [1]:
import pandas as pd
import numpy as np
import pandahouse as ph
import seaborn as sns

## 1. ##

Посчитать **OnTime** - Метрика своевременности доставки. (Доставили в плановую дату доставки или нет), в %

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада, по таймслотам;

Для начала считываем все данные, проверяем количество строк, типы данных, наличие пропущенных значений, наличие дубликатов, переводим даты к типу datetime

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database': 'default',
                      'user': 'student',
                      'password': 'dpo_python_2020'
                     }

In [3]:
# "скачиваем" все данные из clickhouse logistics.online
q = '''
    SELECT
        assumeNotNull(date_order) as date_order, 
        number_order,
        assumeNotNull(planned_data_order) AS planned_data_order, 
        timeslot_number, 
        assumeNotNull(result_data_order) AS result_data_order, 
        delivery_type,
        carrier_name,
        seller_ID,
        order_status,
        customer_cluster,
        warehouse_cluster
    FROM {db}.logistics_online
    where isNotNull(date_order)
    '''

logistics = ph.read_clickhouse(query = q, connection = connection_default)
logistics.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
0,2021-5-01,313655114,2021-14-01,1,2021-11-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва
1,2021-3-01,315229153,2021-11-01,1,2021-8-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
2,2021-8-01,3114106095,2021-16-01,1,2021-14-01,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область
3,2021-12-01,31145541407,2021-20-01,1,2021-15-01,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4,2021-16-01,325050136,2021-26-01,1,2021-25-01,Самовывоз,3PL 1,946233,Доставлен,Кластер Санкт-Петербург,Кластер Санкт-Петербург


In [4]:
logistics.shape

(6597, 11)

In [5]:
logistics.loc[logistics.duplicated()]

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
728,2021-16-02,344629456,2021-26-02,1,2021-3-03,Курьер,3PL 1,9504,Доставлен,Кластер Калининград,Кластер Казань


In [6]:
logistics = logistics.drop_duplicates()

In [7]:
# проверяем типы колонок. Все даты у нс object и формата %Y-%d-%m. Поэтому приводим к стандартному формату даты.
# Если дата будет заведена неправильно, мы получим NaT
logistics.dtypes

date_order            object
number_order           int64
planned_data_order    object
timeslot_number         int8
result_data_order     object
delivery_type         object
carrier_name          object
seller_ID              int64
order_status          object
customer_cluster      object
warehouse_cluster     object
dtype: object

In [8]:
logistics['date_order'] = pd.to_datetime(logistics['date_order'], format="%Y-%d-%m", errors='coerce')
logistics['planned_data_order'] = pd.to_datetime(logistics['planned_data_order'], format="%Y-%d-%m", errors='coerce')
logistics['result_data_order'] = pd.to_datetime(logistics['result_data_order'], format="%Y-%d-%m", errors='coerce')

В процессе построения визуализации, обнаружилось, что фактические даты доставки ряда заказов меньше, чем даты самих заказов. Предполагаю, что произошла ошибка в логировании данных. Для корректного расчета метрики, эти данные не буду использовать. 

In [9]:
logistics = logistics.query('result_data_order>=date_order').sort_values('date_order')
logistics.shape

(4180, 11)

In [10]:
# для расчета метрики OnTime возьмем только заказы, у которых есть данные о плановой и фактической дате доставки
logistics_ontime = logistics.dropna()

# создадим колонку со значением True, если плановая дата доставки совпадает с фактической, и False - если наоборот
logistics_ontime['delivered_ontime'] = (logistics_ontime.planned_data_order == logistics_ontime.result_data_order)

logistics_ontime.delivered_ontime = logistics_ontime.delivered_ontime.astype('int')
logistics_ontime.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,delivered_ontime
5930,2021-01-01,314096365,2021-01-09,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,0
2234,2021-01-01,31211414196,2021-01-12,1,2021-01-10,Самовывоз,3PL 1,301202,Доставлен,Кластер Москва,Кластер Москва,0
4144,2021-01-01,3141410151,2021-01-10,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Москва,Кластер Московская область,0
5388,2021-01-01,3017014102,2021-01-05,1,2021-01-11,Курьер,Самодоставка,833469,Доставлен,Кластер Краснодар,Кластер Екатеринбург,0
1281,2021-01-01,315409997,2021-01-12,1,2021-01-11,Самовывоз,3PL 2,41958,Доставлен,Кластер Московская область,Кластер Казань,0


In [11]:
# выгрузим дф logistics_ontime для визуализации в Tableau
logistics_ontime.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ivanova-22/final_project/logistics_ontime.csv')

В качестве "оси" для метрики возьмем плановую дату доставки. Будем считать, сколько процентов доставок осуществились в обещанную дату, в каждую плановую дату доставки.

Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes

## 2. 

Посчитать **PromisedClick2Delivery** - метрика обещанного срока доставки (от даты заказа до первой плановой даты доставки), в сутках.

 - Необходима динамика по дням, неделям, месяцам;

 - Необходимо сравнение перевозчиков;

 - Необходимо понимание уровня метрики из кластера в кластер

Для расчета этой метрики не нужно дропать все None. Отфильтруем дф, чтобы в нем остались только данные по первым плановым датам доставки:
- отсортируем по возрастанию плановой даты доставки, чтобы на первом месте была первая
- дропнем None в дате заказа и плановой дате заказа
- дропнем дубликаты по номеру заказа, оставив только первую строку. Т.о. мы получим только строки, по которым можем сравнить дату заказа и Первую плановую дату доставки
- выгрузим файл для визуализации в Tableau

In [14]:
PromisedClick2Delivery = logistics.sort_values('planned_data_order')\
    .dropna(subset =['date_order', 'planned_data_order'])\
    .drop_duplicates(subset='number_order', keep='first')

In [15]:
PromisedClick2Delivery.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ivanova-22/final_project/PromisedClick2Delivery.csv')

Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes

## 3 ##

Посчитать **Click2Delivery** - метрика фактического срока доставки (от даты заказа до момента вручения), в сутках.

Необходимо сравнение с метрикой PromisedClick2Delivery;

Необходимо понимание кластеров, где заказы доставляются быстрее, чем мы обещаем клиентам

На визуализации график метрики в конце резко падает до 0. Предполагаю, что это связано с недостаточными данными. Потому что это конец нашего дф, заказов после 2021-03-12 очень мало, и конкретно эти заказы доставили очень быстро.

In [67]:
PromisedClick2Delivery.query('date_order>"2021-03-12"').sort_values('date_order')

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster
6163,2021-03-13,3514707550,2021-03-17,1,2021-03-15,Курьер,3PL 1,9504,Доставлен,Кластер Санкт-Петербург,Кластер Казань
5921,2021-03-13,360141414569,2021-03-17,1,2021-03-14,Самовывоз,3PL 1,64865,Доставлен,Кластер Москва,Кластер Северо-Запад
2901,2021-03-13,3514512314,2021-03-17,1,2021-03-15,Самовывоз,3PL 2,41958,Доставлен,Кластер Северо-Запад,Кластер Казань
4138,2021-03-13,361555649,2021-03-17,1,2021-03-14,Самовывоз,3PL 2,588332,Доставлен,Кластер Казань,Кластер Центр
3740,2021-03-13,361654667,2021-03-17,1,2021-03-14,Самовывоз,3PL 1,301202,Доставлен,Кластер Санкт-Петербург,Кластер Москва
4053,2021-03-13,359295202,2021-03-19,1,2021-03-15,Самовывоз,3PL 2,19923,Доставлен,Кластер Казань,Кластер Московская область
746,2021-03-13,362110554,2021-03-18,1,2021-03-15,Самовывоз,3PL 1,51943,Доставлен,Кластер Московская область,Кластер Краснодар
1963,2021-03-13,3619149137,2021-03-18,1,2021-03-13,Самовывоз,3PL 1,301202,Доставлен,Кластер Московская область,Кластер Москва
589,2021-03-13,36014141047,2021-03-19,1,2021-03-14,Самовывоз,3PL 1,833469,Доставлен,Кластер Санкт-Петербург,Кластер Екатеринбург
6071,2021-03-13,35141440039,2021-03-18,1,2021-03-14,Самовывоз,3PL 1,833469,Доставлен,Кластер Казань,Кластер Екатеринбург


Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes

## 4 ##

Посчитать **GMV Accepted** - метрика совокупного объема продаж (считается по заказам, оформленным в указанный день), в рублях.

Необходимо видеть изменение метрики по дням;

Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада

Для начала считываем все данные, проверяем количество строк, типы данных, наличие пропущенных значений, наличие дубликатов

In [16]:
q1 = '''
    SELECT 
        number_order,
        product_ID,
        product_price,
        amount 
    FROM default.product_online 
'''

products = ph.read_clickhouse(query = q1, connection = connection_default)
products.head()

,number_order,product_ID,product_price,amount
0,313655114,983587801,5512.44,1
1,313655114,983587158,4070.35,2
2,313655114,983582669,3591.97,1
3,315229153,983589424,4433.23,3
4,315229153,983587827,2862.50,2


In [17]:
products.shape

(12233, 4)

In [18]:
products.loc[products.duplicated()]

,number_order,product_ID,product_price,amount


In [19]:
products.isna().sum()

number_order     0
product_ID       0
product_price    0
amount           0
dtype: int64

In [20]:
products.dtypes

number_order       int64
product_ID         int64
product_price    float64
amount             int16
dtype: object

In [21]:
# посчитаем выручку по каждому заказу
products['price_in_order'] = products.product_price*products.amount

products_dash = products.groupby('number_order', as_index=False).agg({'price_in_order': 'sum'})

# Удалим из таблицы с логистическими данными все дубликаты по полю "number_order"
# Объединим таблицу с датами заказов и таблицу с выручкой по каждому заказу
GMV_table=logistics.drop_duplicates('number_order')\
    .merge(products_dash, on='number_order', how='left')

In [22]:
# выгрузим полученный дф для визуализации метрики
GMV_table.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ivanova-22/final_project/GMV_table.csv')

Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes

## 5

Посчитать **Return Rate** - доля возвратов (отношение количества возвратов к заказам в этот день), в %.

 - Необходимо видеть изменение метрики по дням;

 - Необходимы разрезы по перевозчикам, по кластерам склада и клиента

Для начала считываем все данные, проверяем количество строк, типы данных, наличие пропущенных значений, наличие дубликатов.

In [38]:
q2 = '''
SELECT 
    number_order,
    product_ID,
    amount_return,
    return_date 
from  default.returns_online
'''

returns = ph.read_clickhouse(query = q2, connection = connection_default)
returns.head(10)

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
1,31014374140,983586554,1,2021-07-01
2,3147145173,983589156,1,2021-11-01
3,3141464669,983589362,1,2021-11-01
4,3131450699,983589729,3,2021-12-01
5,3131450699,983589729,3,2021-12-01
6,315774746,983585035,1,2021-12-01
7,315195519,983587637,1,1970-01-01
8,3155331412,983580141,1,1970-01-01
9,3155331412,983580141,1,1970-01-01


In [39]:
returns.shape

(496, 4)

In [40]:
returns.dtypes

number_order              int64
product_ID                int64
amount_return              int8
return_date      datetime64[ns]
dtype: object

В процессе подготовки данных для расчета метрики я увидела, что формат даты возврата %Y-%d-%m. Я поняла это, когда первый раз объединила таблицы возвратов и логистики. Разница между возвратом и получением заказа была более 200 дней. Это не реально.

Поэтому в начале обработки данных я привожу формат даты возврата к формату, который использую в таблице логистики: %Y-%m-%d

In [41]:
# меняем местами день и месяц, но функция форматирует в тип "строка"
returns.return_date = returns.return_date.dt.strftime('%Y-%d-%m')

# форматируем обратно в дату, но уже с правильным порядком %Y-%m-%d
returns['return_date'] = pd.to_datetime(returns['return_date'], format="%Y-%m-%d", errors='coerce')

In [42]:
# проверяем наличие дубликатов и дропаем их
returns.loc[returns.duplicated()]

,number_order,product_ID,amount_return,return_date
5,3131450699,983589729,3,2021-01-12
9,3155331412,983580141,1,1970-01-01
16,3114571009,983586796,3,1970-01-01
19,312396246,983582111,1,1970-01-01
22,312396246,983588915,1,1970-01-01
...,...,...,...,...
488,35144201422,983588149,1,1970-01-01
489,3600535146,983582199,4,1970-01-01
490,3624014609,983589024,1,1970-01-01
493,35714135014,983587402,2,1970-01-01


In [43]:
returns = returns.drop_duplicates()

In [44]:
returns.shape

(396, 4)

In [45]:
returns_unique = returns.groupby('number_order', as_index=False).agg({'return_date': 'min'})

В дф с заказами почти половина строк оказалась с некорректной unix датой: 1970-01-01. Для расчета данной метрики нам важен лишь сам факт возврата. Поэтому предположу, что произошла ошибка логирования данных, и факт возврата был, но дата записалась некорректно.

В качестве дополнения, если будут необходимы даты возвратов - можно посчитать средний взвешенный срок заказа по остальным товарам и прибавить его к дате получения. Дату "1970-01-01" заменить на получившуюся расчетную дату. 

In [46]:
returns.query('return_date=="1970-01-01"')

,number_order,product_ID,amount_return,return_date
0,3031410942,983588011,1,1970-01-01
7,315195519,983587637,1,1970-01-01
8,3155331412,983580141,1,1970-01-01
10,312396246,983582111,1,1970-01-01
11,312396246,983583606,1,1970-01-01
...,...,...,...,...
485,360670111,983587562,1,1970-01-01
487,36114714697,983581806,1,1970-01-01
491,35714135014,983587402,2,1970-01-01
492,35714135014,983584567,3,1970-01-01


In [59]:
# убираем из логистической дф повторную инфо о доставке заказов, оставляем только последнюю запись
# объединяем дф с возвратами и логистикой по номеру заказа
return_calculation = logistics.drop_duplicates('number_order', 'last')\
    .merge(returns_unique, on='number_order', how='left')

# добавляем колонку, которая будет показывать имел ли место факт возврата - 1, или нет - 0
return_calculation['return_event'] = return_calculation.return_date.notna().astype('int')

In [60]:
# выгрузим дф для построения визуализации в Tableau
return_calculation.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ivanova-22/final_project/return_calculation.csv')

---------------------------------------------------------------------------------------------------------------------

В качестве дополнения - расчет оценочной даты возврата для строк, где дата записалась некорректно

In [33]:
# считаем сколько дней до возврата было по каждому заказу
return_calculation['days_to_return'] = return_calculation.return_date-return_calculation.result_data_order

# выбираем только положительные значения (дата возврата 1970-01-01 будет давать большое отрицательное число дней)
# группируем по количеству дней до возврата и считаем сколько было возвратов для каждого числа дней
WA_return_days = return_calculation.query('days_to_return>"0 days"')\
    .groupby('days_to_return', as_index=False).agg({'number_order': 'count'})\
    .rename(columns={'number_order': 'number_of_returns'})
WA_return_days

,days_to_return,number_of_returns
0,1 days,44
1,2 days,38
2,3 days,29


In [34]:
# считаем среднее взвешенное число дней до возврата
WA_return_days['WA'] = WA_return_days.days_to_return*WA_return_days.number_of_returns


WA=WA_return_days.WA.sum()/WA_return_days.number_of_returns.sum()
WA=WA.round('1d') #округлим результат до одного дня
WA

Timedelta('2 days 00:00:00')

In [35]:
# рассчитаем дату предполагаемого возврата для строк с ошибкой
df_1970 = return_calculation.query('return_date=="1970-01-01"')
df_1970.return_date = df_1970.result_data_order+WA
df_1970.head()

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,return_date,days_to_return
27,2021-01-03,31526147314,2021-01-12,1,2021-01-15,Самовывоз,3PL 1,301202,Доставлен,Кластер Московская область,Кластер Москва,2021-01-17,-18642 days
66,2021-01-04,3155331412,2021-01-14,1,2021-01-12,Самовывоз,3PL 1,855433,Доставлен,Кластер Центр,Кластер Центр,2021-01-14,-18639 days
85,2021-01-05,3114477960,2021-01-16,1,2021-01-15,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,2021-01-17,-18642 days
100,2021-01-06,312396246,2021-01-12,1,2021-01-13,Курьер,Самодоставка,43684,Доставлен,Кластер Санкт-Петербург,Кластер Казань,2021-01-15,-18640 days
112,2021-01-06,314452924,2021-01-14,1,2021-01-17,Самовывоз,3PL 1,301202,Доставлен,Кластер Краснодар,Кластер Москва,2021-01-19,-18644 days


In [36]:
# сформируем дф без строк "1970-01-01" и соединим его с дф с предполагаемыми сроками возврата 
real_return = return_calculation.query('return_date!="1970-01-01"')
result_returns = pd.concat([real_return, df_1970])
result_returns = result_returns.drop(columns='days_to_return')
result_returns.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,return_date
0,2021-01-01,314096365,2021-01-09,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,NaT
1,2021-01-01,31211414196,2021-01-12,1,2021-01-10,Самовывоз,3PL 1,301202,Доставлен,Кластер Москва,Кластер Москва,NaT
2,2021-01-01,3141410151,2021-01-10,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Москва,Кластер Московская область,NaT
3,2021-01-01,3017014102,2021-01-05,1,2021-01-11,Курьер,Самодоставка,833469,Доставлен,Кластер Краснодар,Кластер Екатеринбург,NaT
4,2021-01-01,315409997,2021-01-12,1,2021-01-11,Самовывоз,3PL 2,41958,Доставлен,Кластер Московская область,Кластер Казань,NaT


Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes

## 6

Посчитать **GMV D-R** - метрика, отражающая совокупный объем доставленных заказов за вычетом возвратов, в рублях.

 - Необходимо видеть изменение метрики по дням (для всех дней, которые были раньше, чем дата расчета метрики);

 - Необходимы разрезы по перевозчикам, по типу доставки, по кластерам склада

In [68]:
# рассчитаем сумму возврата для каждого заказа
# для этого сформируем дф с ценами продуктов
prices = products.groupby('product_ID', as_index=False).agg({'product_price':'min'})

In [76]:
# добавим инфо о ценах продуктов в таблицу с возвратами, очищенную от дубликатов
return_price = returns.merge(prices, on='product_ID', how='left')

# посчитаем сумму к возврату для каждого продукта и для заказа в целом
return_price['return_price'] = return_price.amount_return*return_price.product_price
return_price = return_price.groupby('number_order', as_index=False).agg({'return_price': 'sum'})

# объединим дф с информацией о суммах заказов с дф с данными о суммах возвратов по заказам
GMV_table_D_R = GMV_table.merge(return_price, on='number_order', how='left')

In [86]:
# заменим NaN на 0 и посчитаем итоговую выручку от каждого заказа
GMV_table_D_R.return_price = GMV_table_D_R.return_price.fillna(0)
GMV_table_D_R['result_from_order'] = GMV_table_D_R.price_in_order-GMV_table_D_R.return_price
GMV_table_D_R.head()

,date_order,number_order,planned_data_order,timeslot_number,result_data_order,delivery_type,carrier_name,seller_ID,order_status,customer_cluster,warehouse_cluster,price_in_order,return_price,result_from_order
0,2021-01-01,314096365,2021-01-09,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Московская область,Кластер Московская область,26002.44,0.0,26002.44
1,2021-01-01,31211414196,2021-01-12,1,2021-01-10,Самовывоз,3PL 1,301202,Доставлен,Кластер Москва,Кластер Москва,33194.08,0.0,33194.08
2,2021-01-01,3141410151,2021-01-10,1,2021-01-08,Курьер,Самодоставка,40560,Доставлен,Кластер Москва,Кластер Московская область,2907.72,0.0,2907.72
3,2021-01-01,3017014102,2021-01-05,1,2021-01-11,Курьер,Самодоставка,833469,Доставлен,Кластер Краснодар,Кластер Екатеринбург,6054.00,0.0,6054.00
4,2021-01-01,315409997,2021-01-12,1,2021-01-11,Самовывоз,3PL 2,41958,Доставлен,Кластер Московская область,Кластер Казань,21017.96,0.0,21017.96


In [87]:
# выгрузим дф для построения визуализации в Tableau
GMV_table_D_R.to_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ivanova-22/final_project/GMV_table_D_R.csv')

Ссылка на дашборд/метрику в Tableau:

https://public.tableau.com/app/profile/valentina.ivanova/viz/ozondashboard/DashboardDeliveryandsalesvolumemetrics?publish=yes